In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/yzxing87/pytorch-deep-video-prior.git

Cloning into 'pytorch-deep-video-prior'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 855 (delta 2), reused 6 (delta 0), pack-reused 844
Receiving objects: 100% (855/855), 101.21 MiB | 39.96 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
%cd pytorch-deep-video-prior

/content/pytorch-deep-video-prior


In [4]:
!cp '../drive/MyDrive/colorization/dataset_colorization_without_bb.zip' .
!cp '../drive/MyDrive/colorization/dataset_colorization.zip' .
!cp '../drive/MyDrive/colorization/dataset_grayscale.zip' .
!cp '../drive/MyDrive/colorization/dataset_prior_0005.zip' .
!cp '../drive/MyDrive/colorization/dataset_prior_0025.zip' .
!cp '../drive/MyDrive/colorization/dataset_prior_0050.zip' .

In [5]:
!unzip 'dataset_colorization_without_bb.zip'
!unzip 'dataset_colorization.zip'
!unzip 'dataset_grayscale.zip'
!unzip 'dataset_prior_0005.zip'
!unzip 'dataset_prior_0025.zip'
!unzip 'dataset_prior_0050.zip'

Streaming output truncated to the last 5000 lines.
  inflating: dataset_prior_0025/horsejump-high/predictions_00004.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_minor_00019.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_main_00013.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_main_00000.jpg  
  inflating: dataset_prior_0025/horsejump-high/predictions_00006.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_minor_00042.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_minor_00005.jpg  
  inflating: dataset_prior_0025/horsejump-high/predictions_00047.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_minor_00043.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_minor_00016.jpg  
  inflating: dataset_prior_0025/horsejump-high/out_minor_00040.jpg  
  inflating: dataset_prior_0025/horsejump-high/predictions_00032.jpg  
  inflating: dataset_prior_0025/horsejump-high/predictions_00023.jpg  
  inflating: dataset_prior_0025/horsejump-hi

In [6]:
!wget https://graphics.ethz.ch/Downloads/Data/Davis/DAVIS-data.zip
!unzip DAVIS-data.zip 

Streaming output truncated to the last 5000 lines.
  inflating: DAVIS/JPEGImages/480p/elephant/00052.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00031.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00019.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00068.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00078.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00032.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00055.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00023.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00053.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00018.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00004.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00020.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00067.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00026.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00057.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00071.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00028.jpg

In [7]:
import os; val_set=os.listdir('dataset_colorization'); print(val_set)

['libby', 'horsejump-high', 'car-roundabout', 'dog', 'drift-chicane', 'scooter-black', 'parkour', 'car-shadow', 'cows', 'camel', 'bmx-trees', 'kite-surf', 'breakdance', 'blackswan', 'paragliding-launch', 'drift-straight', 'dance-twirl', 'soapbox', 'goat', 'motocross-jump']


## SSIM

In [8]:
def load_image_with_pad(path):
    if(path[-3:] == 'dng'):
        import rawpy
        with rawpy.imread(path) as raw:
            img = raw.postprocess()
    elif(path[-3:]=='bmp' or path[-3:]=='jpg' or path[-3:]=='png'):
        img = cv2.imread(path)[:,:,::-1]
        org_h,org_w = img.shape[:2]
        h = org_h // 32 * 32
        w = org_w // 32 * 32
        img = img[:h, :w, :]
    else:
        img = (255*plt.imread(path)[:,:,:3]).astype('uint8')
        org_h,org_w = img.shape[:2]
        h = org_h // 32 * 32
        w = org_w // 32 * 32
        img = img[:h, :w, :]
    return img

In [13]:
from tqdm import tqdm
import torch
import cv2
import numpy as np
from skimage.measure import compare_ssim

original="DAVIS/JPEGImages/480p"
colorization="dataset_colorization"

def calculate_ssim_dir(original, test):
  dists={}
  for folder in val_set:
    subfolder = os.path.join(original, folder)
    for filename in tqdm(sorted(os.listdir(subfolder))):
      img_path0 = os.path.join(original, os.path.join(folder, filename))
      img_path1 = os.path.join(test, os.path.join(folder, filename))

      img0 = load_image_with_pad(img_path0)
      img1 = load_image_with_pad(img_path1)

      dist = compare_ssim(img0, img1, multichannel=True)

      if folder not in dists:
        dists[folder] = [dist] 
      else:
        dists[folder].append(dist)
  return dists

In [14]:
import cv2
original="DAVIS/JPEGImages/480p"
tests = ['dataset_colorization', 'dataset_colorization_without_bb']
output = {test: calculate_ssim_dir(original, test) for test in tests}

100%|██████████| 40/40 [00:06<00:00,  6.02it/s]


In [15]:
for folder in val_set:
  for dataset in ['dataset_prior_0005', 'dataset_prior_0025', 'dataset_prior_0050']:
    subfolder = os.path.join(dataset, folder)
    for filename in os.listdir(subfolder):
      if filename.startswith('out_main_'):
        src = os.path.join(subfolder, filename)
        dst = os.path.join(subfolder, filename.split('_')[-1])
        os.system('mv {} {}'.format(src, dst))

In [18]:
def calculate_ssim_dir_with_resize(original, test):
  dists={}
  for folder in val_set:
    subfolder = os.path.join(original, folder)
    for filename in tqdm(sorted(os.listdir(subfolder))):
      img_path0 = os.path.join(original, os.path.join(folder, filename))
      img_path1 = os.path.join(test, os.path.join(folder, filename))

      img0 = load_image_with_pad(img_path0)
      img1 = load_image_with_pad(img_path1)

      dist = compare_ssim(img0, img1, multichannel=True)

      if folder not in dists:
        dists[folder] = [dist] 
      else:
        dists[folder].append(dist)
  return dists

In [19]:
original = "DAVIS/JPEGImages/480p"
tests = ['dataset_prior_0005', 'dataset_prior_0025', 'dataset_prior_0050']
for test in tests:
  output[test] = calculate_ssim_dir_with_resize(original, test)

  0%|          | 0/49 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  if sys.path[0] == '':
100%|██████████| 40/40 [00:06<00:00,  6.29it/s]


In [20]:
values = {}
for key in ['dataset_colorization', 'dataset_prior_0005', 'dataset_prior_0025', 'dataset_prior_0050']:
  values[key] = []
  for label in val_set:
    values[key].append(np.mean(output[key][label]))
  print('{}: {}'.format(key, np.mean(values[key])))

dataset_colorization: 0.959338379417409
dataset_prior_0005: 0.8749546967299805
dataset_prior_0025: 0.919597458205352
dataset_prior_0050: 0.923896031000868
